In [1]:
import pandas as pd
from globals import YELP_DIR
import json
import os

YELP_DIR = "/Volumes/Forster Neu/Masterarbeit Data/yelp_dataset/"
ORIGINAL_DATA_FOLDER = "/Users/andreaforster/dev/thesis/CAPRI/Data/Yelp/"

This Yelp dataset was used in the following script: https://www.yelp.com/dataset

In [2]:
business_df = pd.read_json(YELP_DIR + "yelp_academic_dataset_business.json", lines=True)

In [3]:
checkin_df = pd.read_json(YELP_DIR + "yelp_academic_dataset_checkin.json", lines=True)

In [4]:
def open_big_json(file_path):
    data = []

    # Open the file and read it line by line
    with open(file_path, 'r') as file:
        for line in file:
            # Parse the JSON data and append to the list
            data.append(json.loads(line))

    # Create a DataFrame from the list of records
    df = pd.DataFrame(data)

    # Print the first few rows of the DataFram

    return df

In [5]:
user_df = open_big_json(YELP_DIR + "yelp_academic_dataset_user.json")
review_df = open_big_json(YELP_DIR + "yelp_academic_dataset_review.json")

In [15]:
# def review_binary(row):
#     if row['stars'] >= 4:
#         return 1
#     else:
#         return 0

In [16]:
# filtering out users and businesses with less than 10 reviews

business_counts = review_df["business_id"].value_counts()
mask = review_df['business_id'].map(business_counts) >= 10
review_df_filtered = review_df.loc[mask]

counts = review_df_filtered['user_id'].value_counts()
mask = review_df_filtered['user_id'].map(counts) >= 10
review_df_filtered = review_df_filtered.loc[mask]


# dropping text column
review_df_filtered = review_df_filtered.drop("text", axis=1)
# turning reviews (stars) into implicit feedback (binary)
#review_df_filtered["stars_bin"] = review_df_filtered.apply(review_binary, axis=1)

# getting the ids of the remaining users and businesses
users_with_more_than_10_reviews = review_df_filtered['user_id'].unique()
businesses_with_more_than_10_reviews = review_df_filtered['business_id'].unique()

# filtering out the other users and businesses from the original dataframes
user_df_filtered = user_df[user_df['user_id'].isin(users_with_more_than_10_reviews)]
business_df_filtered = business_df[business_df['business_id'].isin(businesses_with_more_than_10_reviews)]


# getting the highest number of reviews for a business to normalize the popularity of businesses
max_num_reviews = business_df_filtered["review_count"].max()
business_df_filtered["popularity"] = business_df_filtered["review_count"] / max_num_reviews 

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_52678/2360509686.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  business_df_filtered["popularity"] = business_df_filtered["review_count"] / max_num_reviews


In [17]:
print(f"Lengh of review_df_filtered: {len(review_df_filtered)} vs. length of review_df: {len(review_df)}")
print(f"Length of user_df_filtered: {len(user_df_filtered)} vs. length of user_df: {len(user_df)}")
print(f"Length of business_df_filtered: {len(business_df_filtered)} vs. length of business_df: {len(business_df)}")

Lengh of review_df_filtered: 3111228 vs. length of review_df: 6990280
Length of user_df_filtered: 111434 vs. length of user_df: 1987897
Length of business_df_filtered: 100184 vs. length of business_df: 150346


In [18]:
review_df_filtered["business_id"].value_counts()

business_id
ytynqOUb3hjKeJfRj5Tshw    2856
_ab50qdWOk0DdB6XOrBitw    1996
9PZxjhTIU7OgPIzuGi89Ew    1875
ctHjyadbDQAtUFfkcAFEHw    1817
GXFMD0Z4jEVZBCsbPf4CTQ    1770
                          ... 
Ew1jBB6kHoQUhuqqXrJa-A       1
gmmRfSIX0ixOCenBmGwlBQ       1
Zo9n9WMGsEBJBL18tP1ekQ       1
wqBCl5-yZQ2z1Bm4tL-JxA       1
uY96VNf906Az5p0iA54xYQ       1
Name: count, Length: 100184, dtype: int64

In [19]:
review_df_filtered["user_id"].value_counts()

user_id
_BcWyKQL16ndpBdggh2kNA    2772
ET8n-r7glWYqZhuR6GcdNw    1601
Xw7ZjaGfr0WNVt6s_5KZfA    1600
0Igx-a1wAstiBDerGxXk2A    1528
-G7Zkl1wIWBBmD0KRy_sCw    1509
                          ... 
0GxT4IW5yLRZOksENl4MqA      10
sjz1ERGjiTIP-kLOTrshVg      10
Kf1n2U67z8485JR5VWkCwQ      10
tfANULks7555uJ7UznUqpg      10
VcMZN1b6QbdoAHmvc-2ueg      10
Name: count, Length: 111434, dtype: int64

In [20]:
def user_popularity_calculator(user_df, review_df, business_df):
    review_business_df = review_df.merge(business_df, on="business_id", how="inner")
    average_popularity_per_user = review_business_df.groupby('user_id')['popularity'].mean().reset_index()
    user_df = pd.merge(user_df, average_popularity_per_user, on='user_id', how='left')

    user_df = user_df.loc[user_df["popularity"] > 0]

    #user_df = user_df.sort_values(by='popularity', ascending=False)

    # Calculate thresholds for high, medium, and low popularity
    threshold_low = user_df['popularity'].quantile(0.2)
    threshold_med = user_df['popularity'].quantile(0.8)
    
    # Filter users based on thresholds
    high_pop_user_df = user_df[user_df['popularity'] >= threshold_med]
    medium_pop_user_df = user_df[(user_df['popularity'] >= threshold_low) & (user_df['popularity'] < threshold_med)]
    low_pop_user_df = user_df[user_df['popularity'] < threshold_low]


    


    return high_pop_user_df, medium_pop_user_df, low_pop_user_df
    

In [21]:
high_pop_user_df, medium_pop_user_df, low_pop_user_df = user_popularity_calculator(user_df_filtered, review_df_filtered, business_df_filtered)

In [22]:
def data_sample_maker(high_pop_user_df, medium_pop_user_df, low_pop_user_df, review_df, business_df):
    high_pop_user_df_sample = high_pop_user_df.sample(n=1000, random_state=123)
    medium_pop_user_df_sample = medium_pop_user_df.sample(n=1000, random_state=123)
    low_pop_user_df_sample = low_pop_user_df.sample(n=1000, random_state=123)

    unique_users = list(set(high_pop_user_df_sample["user_id"].tolist() + medium_pop_user_df_sample["user_id"].tolist() + low_pop_user_df_sample["user_id"].tolist()))

    review_df_sample = review_df.loc[review_df["user_id"].isin(unique_users)]

    business_df_sample = business_df.loc[business_df["business_id"].isin(review_df_sample["business_id"].tolist())]

    return review_df_sample, business_df_sample, high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample

    

In [23]:
review_df_sample, business_df_sample, high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample = data_sample_maker(high_pop_user_df, medium_pop_user_df, low_pop_user_df, review_df_filtered, business_df_filtered)

In [24]:
user_df_sample = pd.concat([high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample])

In [25]:
review_df_sample['checkin_count'] = review_df_sample.groupby(['user_id', 'business_id'])['business_id'].transform('count')


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_52678/4232908792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df_sample['checkin_count'] = review_df_sample.groupby(['user_id', 'business_id'])['business_id'].transform('count')


In [26]:
review_df_sample = review_df_sample.drop(["stars_bin","stars"], axis=1)

In [27]:
review_df_sample

,review_id,user_id,business_id,useful,funny,cool,date,checkin_count
49,cvQXRFLCyr0S7EgFb4lZqw,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,3,1,1,2009-10-14 01:15:04,1
74,EZarjNNbO_2yH1Xbizog9g,R_W9WlKiA56VzVbRzTULQQ,pR8u8hXf1vvzoAGOoKHQqQ,1,0,0,2016-08-25 17:17:46,1
101,pl5AjpEcFxFTltkBvHjsRA,YjS6MDNwGbueb5WtALIJ2A,EBn3U4mpnIRLIy2lKuilRQ,0,0,0,2012-11-06 18:54:49,1
135,WHBZtv1ZabwX0r-alDjA3g,x1qKiRxnPLf3JvOsUAsv_Q,GgGfhKUGusCbP-rvjHiS3A,0,0,0,2016-01-17 18:07:08,1
243,uwPhE21CZLlp1kkSMPvvYw,5OnQqP3q2_9auNdDKpLdsw,WKMJwqnfZKsAae75RMP6jA,0,0,0,2012-11-06 23:35:15,1
...,...,...,...,...,...,...,...,...
6990047,jsBz-uJy6M--L8ogPG52dA,bJ5FtCtZX3ZZacz2_2PJjA,mBgaPljP3OYkl_vGKTyFNw,4,1,4,2012-03-16 05:37:03,1
6990196,EQQWfhD7l2X767XHZykdDA,hVXj8lnaTIMLTkD_yjSj6A,PlMXeyXhy1FcnvOSLBGFQQ,2,0,0,2020-09-11 19:03:01,1
6990234,oVH6oK8G_FTYiaglrIk09A,piIljcy0fhEW6IhB1eyrFA,oBNrLz4EDhiscSlbOl8uAw,0,1,0,2021-11-27 02:19:59,1
6990239,me7QTotYCOjWNVA8bzN1eg,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,3,1,3,2007-07-27 20:12:11,1


In [28]:
def data_saver(df, filename, framework):
    if not os.path.exists(YELP_DIR + "processed_data_" + framework):
        os.makedirs(YELP_DIR + "processed_data_" + framework)
    
    df.to_csv(YELP_DIR + "processed_data_" + framework + "/" + filename + ".csv")
    print("Data saved as " + framework + filename + ".csv")
    


In [29]:
def data_saver_recbole(df, framework, suffix):
    
    if not os.path.exists(YELP_DIR + "processed_data_" + framework):
        os.makedirs(YELP_DIR + "processed_data_" + framework)

    df.to_csv(f"{YELP_DIR}processed_data_{framework}/yelp_sample.{suffix}", sep="\t", index=False)

In [30]:
# first of all saving data for cornac
data_saver(review_df_sample, "user_events", "cornac")
data_saver(business_df_sample, "businesses", "cornac")
data_saver(high_pop_user_df_sample, "high_pop_user_sample", "cornac")
data_saver(medium_pop_user_df_sample, "medium_pop_user_sample", "cornac")
data_saver(low_pop_user_df_sample, "low_pop_user_sample", "cornac")


Data saved as cornacuser_events.csv
Data saved as cornacbusinesses.csv
Data saved as cornachigh_pop_user_sample.csv
Data saved as cornacmedium_pop_user_sample.csv
Data saved as cornaclow_pop_user_sample.csv


In [31]:
# secondly making adjustments to the column names to fit the format in recbole
review_df_sample_recbole = review_df_sample.copy()
review_df_sample_recbole.rename(columns={"user_id":"user_id:token", "business_id":"item_id:token", "checkin_count":"rating:float", "date":"timestamp:float", "useful":"useful:float", "funny":"funny:float", "cool": "cool:float", "review_id": "review_id:token"}, inplace=True)
try:
    review_df_sample_recbole.drop("stars", axis=1, inplace=True)
    review_df_sample_recbole.drop("stars_bin", axis=1, inplace=True)
except Exception as e:
    print(e)

"['stars'] not found in axis"


In [32]:
item_df_sample_recbole = business_df_sample.copy()
item_df_sample_recbole.rename(columns={"business_id":"item_id:token", "name":"item_name_token_seq", "address":"address:token_seq", "city":"city:token_seq", "state":"state:token", "postal_code":"postal_code:token", "latitude":"latitude:float", "longitude":"longitude:float", "stars":"item_stars:float", "review_count":"item_review_count:float", "is_open":"is_open:float", "categories":"categories:token_seq"}, inplace=True)

try:
    item_df_sample_recbole.drop(["popularity", "attributes", "hours"], axis=1, inplace=True)
except Exception as e:
    print(e)

In [33]:
user_df_sample_recbole = user_df_sample.copy()
user_df_sample_recbole.rename(columns={"user_id":"user_id:token", "name":"user_name:token", "review_count":"user_review_count:float", "yelping_since":"yelping_since:float", "useful":"user_useful:float", "funny":"user_funny:float", "cool":"user_cool:float", "elite": "elite:token", "fans":"fans:float", "average_stars":"average_stars:float", "compliment_hot":"compliment_hot:float", "compliment_more":"compliment_more:float", "compliment_profile":"compliment_profile:float", "compliment_cute":"compliment_cute:float", "compliment_list":"compliment_list:float", "compliment_note":"compliment_note:float", "compliment_plain":"compliment_plain:float", "compliment_cool":"compliment_cool:float", "compliment_funny":"compliment_funny:float", "compliment_writer":"compliment_writer:float", "compliment_photos":"compliment_photos:float"}, inplace=True)
try:
    user_df_sample_recbole.drop(["popularity"], axis=1, inplace=True)
except Exception as e:
    print(e)


In [34]:
review_df_sample_recbole = review_df_sample_recbole.drop_duplicates(subset=["user_id:token", "item_id:token"], keep="first")

In [37]:
review_df_sample_recbole

,review_id:token,user_id:token,item_id:token,useful:float,funny:float,cool:float,timestamp:float,rating:float
49,cvQXRFLCyr0S7EgFb4lZqw,ZGjgfSvjQK886kiTzLwfLQ,EtKSTHV5Qx_Q7Aur9o4kQQ,3,1,1,2009-10-14 01:15:04,1
74,EZarjNNbO_2yH1Xbizog9g,R_W9WlKiA56VzVbRzTULQQ,pR8u8hXf1vvzoAGOoKHQqQ,1,0,0,2016-08-25 17:17:46,1
101,pl5AjpEcFxFTltkBvHjsRA,YjS6MDNwGbueb5WtALIJ2A,EBn3U4mpnIRLIy2lKuilRQ,0,0,0,2012-11-06 18:54:49,1
135,WHBZtv1ZabwX0r-alDjA3g,x1qKiRxnPLf3JvOsUAsv_Q,GgGfhKUGusCbP-rvjHiS3A,0,0,0,2016-01-17 18:07:08,1
243,uwPhE21CZLlp1kkSMPvvYw,5OnQqP3q2_9auNdDKpLdsw,WKMJwqnfZKsAae75RMP6jA,0,0,0,2012-11-06 23:35:15,1
...,...,...,...,...,...,...,...,...
6990047,jsBz-uJy6M--L8ogPG52dA,bJ5FtCtZX3ZZacz2_2PJjA,mBgaPljP3OYkl_vGKTyFNw,4,1,4,2012-03-16 05:37:03,1
6990196,EQQWfhD7l2X767XHZykdDA,hVXj8lnaTIMLTkD_yjSj6A,PlMXeyXhy1FcnvOSLBGFQQ,2,0,0,2020-09-11 19:03:01,1
6990234,oVH6oK8G_FTYiaglrIk09A,piIljcy0fhEW6IhB1eyrFA,oBNrLz4EDhiscSlbOl8uAw,0,1,0,2021-11-27 02:19:59,1
6990239,me7QTotYCOjWNVA8bzN1eg,bJ5FtCtZX3ZZacz2_2PJjA,wMQkdK2aNMvq2xoojC98Mw,3,1,3,2007-07-27 20:12:11,1


In [38]:
review_df_sample_recbole["timestamp:float"] = pd.to_datetime(review_df_sample_recbole["timestamp:float"]).apply(lambda x: int(x.timestamp()))

In [42]:
user_df_sample_recbole = user_df_sample_recbole.drop(["friends"], axis=1)

In [43]:
user_df_sample_recbole

,user_id:token,user_name:token,user_review_count:float,yelping_since:float,user_useful:float,user_funny:float,user_cool:float,elite:token,fans:float,average_stars:float,...,compliment_more:float,compliment_profile:float,compliment_cute:float,compliment_list:float,compliment_note:float,compliment_plain:float,compliment_cool:float,compliment_funny:float,compliment_writer:float,compliment_photos:float
55684,k1SkRKGIX2YZu98GoytWsg,Lisa,40,2011-12-05 00:54:47,21,2,5,,0,3.93,...,0,0,0,0,1,0,0,0,1,0
96103,xBqn1EyblQBPrCpSL_iu1A,Bronson,105,2010-09-01 21:27:19,226,78,139,"2018,2019,20,20,2021",9,3.98,...,2,0,0,0,5,8,7,7,5,5
64225,13ZAPxr6DPkzK71_nGNi3w,Carolina,328,2010-03-24 20:24:04,347,91,137,"2014,2015,2016,2017,2018,2019,20,20,2021",15,3.80,...,1,0,0,0,10,3,32,32,4,1
104498,3ZObaHG9jvLKboVUKPKQaA,Becca,319,2011-04-04 22:32:20,575,89,186,"2013,2014,2015,2016,2017,2018,2019,20,20,2021",16,4.21,...,5,1,0,0,7,14,20,20,13,10
18702,4l1qyHWA8iQyFeZl2cnc8g,Alisa,50,2014-05-03 23:49:01,10,5,6,,0,4.22,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110839,PnLZNjlzeCBzWY4sfuOc8Q,Amy,22,2016-09-02 14:57:58,4,1,0,,0,3.87,...,0,0,0,0,1,0,0,0,1,0
30276,MK-IYG8gyI3liNvklNt7DQ,Crystal,14,2013-03-21 16:55:39,11,0,0,,0,4.43,...,0,0,0,0,0,1,0,0,0,0
86187,jBMZpYkoMueVzgO9FSti3w,Tom,10,2018-07-21 19:53:46,5,0,4,,0,3.83,...,0,0,0,0,0,0,0,0,0,0
54198,GjV7x4bM1o838d5XSUk0ww,Greg,39,2014-06-05 14:04:56,35,1,10,,0,4.30,...,0,0,0,0,0,0,0,0,0,0


In [44]:
data_saver_recbole(review_df_sample_recbole, "recbole", "inter")
data_saver_recbole(user_df_sample_recbole, "recbole", "user")
data_saver_recbole(item_df_sample_recbole, "recbole", "item")

In [36]:
# convert time stamp to unix format for temporal splitting

def convert_timestamp_to_unix(input_path,output_path,timestamp_column):
    # Read the CSV file
    df = pd.read_csv(input_path, sep='\t')

    # Convert the timestamp column to UNIX format
    df[timestamp_column] = pd.to_datetime(df[timestamp_column]).apply(lambda x: int(x.timestamp()))

    
    # Save the updated DataFrame to a new CSV file
    df.to_csv(output_path, sep='\t', index=False)


# Define input and output paths
framework = "recbole"
suffix = "inter"
timestamp_column = "timestamp:float"  # replace with the name of your timestamp column

input_path = os.path.join(YELP_DIR, f"processed_data_{framework}/yelp_sample.{suffix}")
output_path = os.path.join(YELP_DIR, f"processed_data_{framework}/yelp_sample_unix.{suffix}")

# Convert timestamps and save the new file
convert_timestamp_to_unix(input_path, output_path,timestamp_column)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x105f59550>>
Traceback (most recent call last):
  File "/Users/andreaforster/dev/thesis/preprocessing/prep_venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [6]:
#lastly, getting the data ready for CAPRI the POI recommendation framework

business_df_sample = pd.read_csv(YELP_DIR + "processed_data_cornac/businesses.csv", index_col=0)
review_df_sample = pd.read_csv(YELP_DIR + "processed_data_cornac/user_events.csv", index_col=0)
high_pop_user_df_sample = pd.read_csv(YELP_DIR + "processed_data_cornac/high_pop_user_sample.csv", index_col=0)
medium_pop_user_df_sample = pd.read_csv(YELP_DIR + "processed_data_cornac/medium_pop_user_sample.csv", index_col=0)
low_pop_user_df_sample = pd.read_csv(YELP_DIR + "processed_data_cornac/low_pop_user_sample.csv", index_col=0)

In [9]:
review_df_sample.sort_values(by="checkin_count", ascending=False)

review_df_sample[""]

,review_id,user_id,business_id,useful,funny,cool,date,checkin_count
1338449,BXQ8clxxE04aFt2rlsOc5Q,Sa6MxEeKd8euwZTsid4Tkg,I220i0Sx3jkkO_1EzLo7QA,0,0,0,2017-04-27 01:48:20,12
1260799,ATne4P-jh4IXcoaAMUJr-g,Sa6MxEeKd8euwZTsid4Tkg,I220i0Sx3jkkO_1EzLo7QA,1,0,0,2018-07-11 00:17:10,12
1308431,vC6ZSrLs7HH1r0-VCv2FyA,Sa6MxEeKd8euwZTsid4Tkg,I220i0Sx3jkkO_1EzLo7QA,0,0,0,2019-07-14 23:38:20,12
1253548,s3audIDspfa65P8UYjMYRQ,Sa6MxEeKd8euwZTsid4Tkg,I220i0Sx3jkkO_1EzLo7QA,0,0,0,2017-10-05 15:59:04,12
1285864,yn10HPu03MF-EITPOymmMA,Sa6MxEeKd8euwZTsid4Tkg,I220i0Sx3jkkO_1EzLo7QA,0,0,0,2016-09-30 19:42:01,12
...,...,...,...,...,...,...,...,...
2413375,tDJteE67q1HWRN6Wj3qT7Q,pBTvkU93L9vaIxvCQOur_Q,p73i2GDzPRZJbYBa8qtpSQ,0,0,0,2013-02-12 23:33:27,1
2413403,QhKurTx5_BnpuMPhRWbFfw,dKRYOegmDF4eJ0PssDsCRw,me3mlZsR26fV89fMw2uR6Q,0,0,0,2015-01-29 18:37:49,1
2413423,7Vkvl3N0isYha2ugO6uusQ,B4KEizUmIP-UzhHjxxS_VA,cE8U_UdSFT4U7rsLqFhZig,0,0,0,2014-06-24 12:11:27,1
2413434,ixKQ9QsFvX5RB_Qx6RfX2A,iCNiJk_p3LPfrZaUwUe1ag,vC2vhaJmX1RpExoHcPOb_A,0,0,0,2018-01-16 18:18:51,1


In [6]:
reference_checkins = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}checkins.txt', sep='\s+', header=None, names=['user_id', 'item_id', 'checkin_timestamp'])
reference_categories = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}poiCategories.txt', sep='\s+', header=None, names=['item_id', 'category_id'])
print(f"Number of users: {len(reference_checkins['user_id'].unique())}, number of POIs: {len(reference_checkins['item_id'].unique())}, number of categories: {len(reference_categories['category_id'].unique())}")
reference_datasize = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}datasize.txt', sep='\s+', header=None, names=['num_users', 'num_items', 'num_categories'])
reference_coos = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}poiCoos.txt', sep='\s+', header=None, names=['item_id', 'latitude', 'longitude'])
reference_socials = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}socialRelations.txt', sep='\s+', header=None, names=['user_id', 'user_id_friend'])


reference_test = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}test.txt', sep='\s+', header=None, names=['user_id', 'item_id', 'no_checkins'])
reference_train = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}train.txt', sep='\s+', header=None, names=['user_id', 'item_id', 'no_checkins'])
reference_tune = pd.read_csv(f'{ORIGINAL_DATA_FOLDER}tune.txt', sep='\s+', header=None, names=['user_id', 'item_id', 'no_checkins'])

print("Length of train, test, tune: ", len(reference_train), len(reference_test), len(reference_tune))
print("... and in percent of the total data: ", len(reference_train) / len(reference_checkins), len(reference_test) / len(reference_checkins), len(reference_tune) / len(reference_checkins))

Number of users: 7135, number of POIs: 16621, number of categories: 595
Length of train, test, tune:  196866 61344 27398
... and in percent of the total data:  0.6524077639658926 0.2032920965160247 0.09079611470308498


In [7]:
reference_datasize

,num_users,num_items,num_categories
0,7135,16621,595


In [87]:
# since in this dataset there are no repeated check-ins, we will test out the binary implicit feedback approach

In [88]:
def convert_to_unix_timestamp(df, column_name):
    """
    Convert a column of timestamps in a DataFrame to Unix timestamps.

    Args:
        df (pd.DataFrame): The DataFrame containing the timestamp column.
        column_name (str): The name of the column with timestamps in "%Y-%m-%d %H:%M:%S" format.

    Returns:
        pd.DataFrame: The DataFrame with an additional column for Unix timestamps.
    """
    # Convert the column to datetime objects
    df[column_name] = pd.to_datetime(df[column_name], format="%Y-%m-%d %H:%M:%S")
    
    # Convert datetime objects to Unix timestamps
    df[f'{column_name}_unix'] = df[column_name].apply(lambda x: x.timestamp())
    
    return df

In [89]:
def unstack_categories(df, column_name):
    """
    Unstack a DataFrame column with comma-separated categories into separate rows.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column with comma-separated categories.

    Returns:
        pd.DataFrame: The DataFrame with each category in a separate row.
    """
    # Split the categories into lists
    df[column_name] = df[column_name].str.split(', ')
    
    # Explode the lists into separate rows
    df = df.explode(column_name)
    
    return df


In [90]:
def factorize_categories(df, column_name):
    """
    Factorize categories in a DataFrame column into unique integer codes.

    Args:
        df (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column with categories.

    Returns:
        pd.DataFrame: The DataFrame with factorized categories.
        pd.Index: The unique categories found in the column.
    """
    # Factorize the categories
    df[column_name + '_codes'], unique_categories = pd.factorize(df[column_name])

    return df, unique_categories

In [91]:
# reverting to unix timestamps and only using the necessary columns
review_df_sample = convert_to_unix_timestamp(review_df_sample, "date")
#ALMOST FINAL - STARS_BIN MUST BE REMOVED FOR checkins.txt AND date_unix FOR train/test/tune.txt
checkins_capri_min = review_df_sample[["user_id", "business_id", "date_unix"]]
checkins_capri_train_test_tune = review_df_sample[["user_id", "business_id", "date_unix", "checkin_count"]]

In [92]:
categories_capri = unstack_categories(business_df_sample, "categories")
categories_capri["category_id"], unique_categories = pd.factorize(categories_capri["categories"])


/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/3621935470.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = df[column_name].str.split(', ')


In [93]:
# filtering out categories with less than 10 POIs
category_counts = categories_capri["category_id"].value_counts()
mask = categories_capri["category_id"].map(category_counts) >= 10
categories_capri = categories_capri.loc[mask]

In [94]:
# FINAL 2
categories_capri_min = categories_capri[["business_id", "category_id"]]

In [95]:
len(checkins_capri_min["user_id"].unique())
len(checkins_capri_min["business_id"].unique())
len(categories_capri_min["category_id"].unique())

print("Length of users: ", len(checkins_capri_min["user_id"].unique()))
print("Length of POIs: ", len(checkins_capri_min["business_id"].unique()))
print("Length of categories_capri_min: ", len(categories_capri_min["category_id"].unique()))


Length of users:  3000
Length of POIs:  35278
Length of categories_capri_min:  645


In [96]:
# FINAL 3
datasize_capri = pd.DataFrame(data={"num_users" : [len(checkins_capri_min["user_id"].unique())], "num_items" : [len(checkins_capri_min["business_id"].unique())], "num_categories" : [len(categories_capri_min["category_id"].unique())]})

In [97]:
# FINAL 4
poi_coos_capri = business_df_sample[["business_id", "latitude", "longitude"]]

In [98]:
user_df_sample = pd.concat([high_pop_user_df_sample, medium_pop_user_df_sample, low_pop_user_df_sample])

In [99]:
social_relations_capri = unstack_categories(user_df_sample, "friends")

In [100]:
# filtering out friends so only those that are in the subset are listed as friends
users = list(social_relations_capri["user_id"].unique())
social_relations_min = social_relations_capri.loc[social_relations_capri["friends"].isin(users)]
# FINAL 5
social_relations_min = social_relations_min[["user_id", "friends"]]

In [101]:
checkins_capri_min['user_id_int'], user_id_map = pd.factorize(checkins_capri_min['user_id'])
checkins_capri_min['business_id_int'], business_id_map = pd.factorize(checkins_capri_min['business_id'])

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/606899870.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkins_capri_min['user_id_int'], user_id_map = pd.factorize(checkins_capri_min['user_id'])
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/606899870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkins_capri_min['business_id_int'], business_id_map = pd.factorize(checkins_capri_min['business_id'])


In [102]:
# FINAL 2.0
checkins_capri_min_int = checkins_capri_min[["user_id_int", "business_id_int", "date_unix"]]

In [103]:
# Create mapping dictionaries
user_id_mapping = {original: i for i, original in enumerate(user_id_map)}
business_id_mapping = {original: i for i, original in enumerate(business_id_map)}

In [104]:
categories_capri_min['business_id_int'] = categories_capri_min['business_id'].map(business_id_mapping)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/1417031790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  categories_capri_min['business_id_int'] = categories_capri_min['business_id'].map(business_id_mapping)


In [105]:
# FINAL 2.0 
categories_capri_min_int = categories_capri_min[["business_id_int", "category_id"]]

In [106]:
poi_coos_capri["business_id_int"] = poi_coos_capri["business_id"].map(business_id_mapping)
# FINAL 2.0
poi_coos_capri_int = poi_coos_capri[["business_id_int", "latitude", "longitude"]]

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/209793819.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poi_coos_capri["business_id_int"] = poi_coos_capri["business_id"].map(business_id_mapping)


In [107]:
social_relations_min['user_id_int'] = social_relations_min['user_id'].map(user_id_mapping)
social_relations_min["friends_int"] = social_relations_min["friends"].map(user_id_mapping)
# FINAL 2.0
social_relations_min_int = social_relations_min[["user_id_int", "friends_int"]]

In [108]:
checkins_capri_train_test_tune['user_id_int'] = checkins_capri_train_test_tune['user_id'].map(user_id_mapping)
checkins_capri_train_test_tune['business_id_int'] = checkins_capri_train_test_tune['business_id'].map(business_id_mapping)

/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/1750645828.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkins_capri_train_test_tune['user_id_int'] = checkins_capri_train_test_tune['user_id'].map(user_id_mapping)
/var/folders/9q/p0b6nwnd5071040x3849mdvc0000gn/T/ipykernel_20615/1750645828.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checkins_capri_train_test_tune['business_id_int'] = checkins_capri_train_test_tune['business_id'].map(business_id_mapping)


In [109]:
# splitting the data into train, test, and tune
checkins_capri_train_test_tune = checkins_capri_train_test_tune.sort_values(by=["user_id_int", "date_unix"])
checkins_capri_train_test_tune = checkins_capri_train_test_tune[["user_id_int", "business_id_int", "checkin_count"]]

# Split the data
train_list = []
val_list = []
test_list = []

for user, group in checkins_capri_train_test_tune.groupby('user_id_int'):
    n = len(group)
    train_end = int(n * 0.65)
    val_end = int(n * 0.80)
    
    train_list.append(group.iloc[:train_end])
    val_list.append(group.iloc[train_end:val_end])
    test_list.append(group.iloc[val_end:])

# Combine lists into DataFrames
train_df = pd.concat(train_list)
val_df = pd.concat(val_list)
test_df = pd.concat(test_list)



# Check the splits

# FINAL 6-8
print("Train Set:")
print(train_df.head())
print("\nValidation Set:")
print(val_df.head())
print("\nTest Set:")
print(test_df.head())


Train Set:
         user_id_int  business_id_int  checkin_count
6303988            0            31917              1
6339649            0            31771              1
3810321            0            19666              1
1220922            0             4027              1
3843050            0            19819              1

Validation Set:
         user_id_int  business_id_int  checkin_count
1217978            0             4240              1
5992674            0            28981              1
4743551            0            24117              1
2435811            0            12435              1
4357977            0            21150              1

Test Set:
         user_id_int  business_id_int  checkin_count
2966116            0            14323              1
1549974            0             7868              1
3545077            0            18107              1
2840083            0            14453              1
1569488            0             8297              1


******

In [110]:
def datasaver_capri(df, filename):
    
    if not os.path.exists(YELP_DIR + "processed_data_capri"):
        os.makedirs(YELP_DIR + "processed_data_capri")
    
    df.to_csv(YELP_DIR + "processed_data_capri/" + filename + ".txt", sep='\t', index=False, header=False)
    print("Data saved as " + filename + ".txt")
    

    


In [111]:
datasaver_capri(checkins_capri_min_int, "checkins")
datasaver_capri(datasize_capri, "dataSize")
datasaver_capri(categories_capri_min_int, "poiCategories")
datasaver_capri(poi_coos_capri_int, "poiCoos")
datasaver_capri(social_relations_min_int, "socialRelations")
datasaver_capri(train_df, "train")
datasaver_capri(val_df, "tune")
datasaver_capri(test_df, "test")

Data saved as checkins.txt
Data saved as dataSize.txt
Data saved as poiCategories.txt
Data saved as poiCoos.txt
Data saved as socialRelations.txt
Data saved as train.txt
Data saved as tune.txt
Data saved as test.txt


## Next To-Do's: 

Potentially changing the hash's to numeric id's

constructing a train, test and tuning set with all the user'id's present in each set 